In [1]:
import sys 
sys.path.append(r"C:\Users\a-sochat\development\Qcodes_Sohail_Clone\Qcodes")

import numpy as np
import time 

import qcodes as qc
from qcodes.instrument.parameter import ManualParameter

from qcodes.sweep import sweep, nest, chain, szip, measurement, setter
from qcodes.sweep.sweep import BaseSweepObject, ParametersTable, wrap_objects, time_trace

C:\Users\a-sochat\AppData\Local\Continuum\Anaconda3\envs\qcodes\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class Printer:
    def __init__(self, sweep_object): 
        self._ind, self._dep = sweep_object.parameter_table.flatten()
        self._symbols_list = sweep_object.parameter_table.symbols_list()
        
    def __enter__(self): 
        header_ind = "\t".join(["{} [{}]".format(*i) for i in self._ind.items()])
        header_dep = "\t".join(["{} [{}]".format(*i) for i in self._dep.items()])
        sep = " | "
        print((header_ind + sep + header_dep).strip(sep))
        
        return self
    
    def __exit__(self, type, value, traceback): 
        pass

    def __call__(self, result):
        print(" " + "\t ".join([str(result[ip]) for ip in self._symbols_list]))

In [3]:
x = ManualParameter("x", unit="V")
y = ManualParameter("y", unit="V")

m = ManualParameter("m", unit="A")
m.get = lambda: x() ** 2

n = ManualParameter("n", unit="A")
n.get = lambda: x() - y() ** 2 + 16

sweep_object = sweep(x, [0, 1, 2])(
    m,
    sweep(y, [0, 1, 2])(
        n  
    )
)

In [4]:
with Printer(sweep_object) as printer:
    for i in sweep_object:   
        printer(i)

x [V]	y [V] | m [A]	n [A]
 0	 None	 0	 None
 0	 0	 None	 16
 0	 1	 None	 15
 0	 2	 None	 12
 1	 None	 1	 None
 1	 0	 None	 17
 1	 1	 None	 16
 1	 2	 None	 13
 2	 None	 4	 None
 2	 0	 None	 18
 2	 1	 None	 17
 2	 2	 None	 14


In [5]:
@measurement([("m3", "H"), ("m4", "H")])
def measurement_function(): 
    meas3 = int(np.random.normal(0, 1) * 10) / 10
    meas4 = int(np.random.normal(-5, 1) * 10) / 10
    return meas3, meas4

In [6]:
sweep_object = sweep(x, [0, 1, 2])(
    m,
    sweep(y, [0, 1, 2])(
        measurement_function  
    )
)

In [7]:
with Printer(sweep_object) as printer:
    for i in sweep_object:   
        printer(i)

x [V]	y [V] | m [A]	m3 [H]	m4 [H]
 0	 None	 0	 None	 None
 0	 0	 None	 -0.3	 -5.6
 0	 1	 None	 0.0	 -3.8
 0	 2	 None	 2.4	 -4.8
 1	 None	 1	 None	 None
 1	 0	 None	 1.7	 -4.6
 1	 1	 None	 -1.1	 -3.8
 1	 2	 None	 -0.3	 -5.6
 2	 None	 4	 None	 None
 2	 0	 None	 -0.9	 -4.2
 2	 1	 None	 -0.2	 -3.5
 2	 2	 None	 1.7	 -4.0


In [8]:
class TimeStamps(BaseSweepObject): 
    def __init__(self):
        super().__init__()
        self._parameter_table = ParametersTable(dependent_parameters=[("time", "s")])
        
    def _setter_factory(self): 
        t0 = time.time()
        while True: 
            t = time.time() - t0
            yield {"time": "{:.3e}".format(t)}

In [9]:
sweep_object2 = szip(TimeStamps(), sweep_object)

In [10]:
with Printer(sweep_object2) as printer:
    for i in sweep_object2:   
        printer(i)

x [V]	y [V] | time [s]	m [A]	m3 [H]	m4 [H]
 0	 None	 0.000e+00	 0	 None	 None
 0	 0	 0.000e+00	 None	 -0.1	 -3.2
 0	 1	 9.992e-04	 None	 0.2	 -3.5
 0	 2	 9.992e-04	 None	 1.7	 -3.5
 1	 None	 9.992e-04	 1	 None	 None
 1	 0	 2.001e-03	 None	 -2.0	 -3.1
 1	 1	 2.001e-03	 None	 -0.2	 -3.0
 1	 2	 2.001e-03	 None	 2.0	 -5.4
 2	 None	 2.001e-03	 4	 None	 None
 2	 0	 2.001e-03	 None	 -0.1	 -4.5
 2	 1	 2.001e-03	 None	 -0.7	 -5.4
 2	 2	 3.000e-03	 None	 1.6	 -5.0


In [11]:
sweep_object3 = time_trace(m, 5, 0.5)

In [12]:
with Printer(sweep_object3) as printer:
    for i in sweep_object3:   
        printer(i)

time [s] | m [A]
 0.0	 4
 0.5007941722869873	 4
 1.0013322830200195	 4
 1.501396894454956	 4
 2.0023789405822754	 4
 2.504150390625	 4
 3.005115270614624	 4
 3.5068256855010986	 4
 4.007554292678833	 4
 4.509374618530273	 4
